In [122]:
# Importando bibliotecas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor

In [123]:
df_livros_vendidos = pd.read_csv("../data/livros_vendidos.csv").drop(columns="preco")

df_livros = df_livros_vendidos[["livro", "genero", "autor", "editora"]]
df_vendas_totais = df_livros_vendidos.groupby("autor")["vendas"].sum().reset_index()
df_vendas_por_livraria = df_livros_vendidos.groupby(["autor", "livraria"])["vendas"].sum().reset_index()

# print(f"df_livros_vendidos:    ", df_livros_vendidos.columns.to_list())
# print(f"df_livros:             ", df_livros.columns.to_list())
# print(f"df_vendas_totais:      ", df_vendas_totais.columns.to_list())
# print(f"df_vendas_por_livraria:", df_vendas_por_livraria.columns.to_list())

In [124]:
# df_livros_vendidos:     ['livro', 'genero', 'autor', 'editora', 'livraria', 'vendas']
# df_livros:              ['livro', 'genero', 'autor', 'editora']
# df_vendas_totais:       ['autor', 'vendas']
# df_vendas_por_livraria: ['autor', 'livraria', 'vendas']

# ===========================================
# 2. Exemplo: unindo as tabelas
# ===========================================
df_base = df_vendas_por_livraria.merge(df_livros, on="autor", how="left")
df_base = df_base.merge(df_vendas_totais, on="autor", how="left", suffixes=("", "_total"))

In [125]:
# ===========================================
# 3. Criação de features
# ===========================================
# Proporção de vendas do autor por livraria
df_base["prop_vendas_livraria"] = df_base["vendas"] / df_base["vendas_total"]

# Popularidade da editora e genero
popularidade_autor = df_base.groupby("autor")["vendas_total"].mean().rename("pop_autor")
popularidade_livraria = df_base.groupby("livraria")["vendas_total"].mean().rename("pop_livraria")
popularidade_editora = df_base.groupby("editora")["vendas_total"].mean().rename("pop_editora")
popularidade_tema = df_base.groupby("genero")["vendas_total"].mean().rename("pop_genero")

df_base = df_base.merge(popularidade_autor, on="autor", how="left")
df_base = df_base.merge(popularidade_livraria, on="livraria", how="left")
df_base = df_base.merge(popularidade_editora, on="editora", how="left")
df_base = df_base.merge(popularidade_tema, on="genero", how="left")
# ===========================================
# 4. Seleção de variáveis
# ===========================================
features = ["autor", "livraria", "genero", "editora", "pop_autor", "pop_livraria", "pop_editora", "pop_genero"]
target = "vendas"

X = df_base[features]
y = df_base[target]
# ===========================================
# 5. Pré-processamento (OneHotEncoder para variáveis categóricas)
# ===========================================
categorical_features = ["autor", "livraria", "genero", "editora"]
numeric_features = ["pop_autor", "pop_livraria", "pop_editora", "pop_genero"]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numeric_features),
    ]
)
# ===========================================
# 6. Modelo (XGBoost)
# ===========================================
model = XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=6, random_state=42)

# Pipeline completo
pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])
# ===========================================
# 7. Treinamento e avaliação
# ===========================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# - MAE (erro médio absoluto):
#       média do erro em unidades de vendas (ex: “em média o modelo erra por 50 cópias”).
# - RMSE (erro quadrático médio):
#       penaliza mais grandes erros — útil para identificar instabilidade.
# - R² (coeficiente de determinação): varia de 0 a 1.
#           0.8 → modelo muito bom.
#       0.5~0.8 → razoável.
#       < 0.5 → precisa de ajuste (talvez falta de variáveis).

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")
# ===========================================
# 8. Previsão para um novo livro
# ===========================================
aut = "João Silva"
liv = "Livraria Fnac"
gen = "Romance"
edi = "Editora Record"
novo_livro = pd.DataFrame(
    [
        {
            "autor": aut,
            "livraria": liv,
            "genero": gen,
            "editora": edi,
            "pop_autor": df_base[df_base["autor"] == aut]["pop_autor"].mean(),
            "pop_livraria": df_base[df_base["livraria"] == liv]["pop_livraria"].mean(),
            "pop_editora": df_base[df_base["editora"] == edi]["pop_editora"].mean(),
            "pop_genero": df_base[df_base["genero"] == gen]["pop_genero"].mean(),
        }
    ]
)

previsao = pipeline.predict(novo_livro)[0]
print(f"📘 Previsão de vendas do novo livro: {previsao:.0f} cópias")

In [ ]:
# 🔎 3. Avaliação local (confiabilidade por previsão)
# Para medir o quanto o modelo está confiante em uma previsão específica, há várias estratégias complementares:
# a) Desvio entre previsões de árvores (XGBoost)
# O XGBoost permite estimar a variabilidade entre árvores — se as árvores divergem muito, a incerteza é alta.

# Número de árvores
ntree = model.n_estimators

# Obter predições de cada árvore individual
predicoes_por_arvore = np.array(
    [
        model.predict(pipeline.named_steps["preprocessor"].transform(novo_livro), iteration_range=(i, i + 1))
        for i in range(ntree)
    ]
).flatten()

media = predicoes_por_arvore.mean()
desvio = predicoes_por_arvore.std()

print(f"Previsão média: {media:.0f}")
print(f"Desvio entre árvores: {desvio:.1f}")

if desvio / media < 0.1:
    print("✅ Alta confiança na previsão")
elif desvio / media < 0.25:
    print("⚠️ Confiança moderada")
else:
    print("❌ Baixa confiança — resultado incerto")

In [ ]:
# b) Distância do novo caso em relação aos dados de treino
# Verifique se o novo livro é similar aos exemplos que o modelo viu:

# Extrair vetores de features (já processadas)
X_train_transf = pipeline.named_steps["preprocessor"].transform(X_train)
novo_transf = pipeline.named_steps["preprocessor"].transform(novo_livro)

# Calcular distância média ao conjunto de treino
distancias = cosine_distances(novo_transf, X_train_transf)
distancia_min = np.min(distancias)

print(f"Distância mínima ao conjunto de treino: {distancia_min:.3f}")

if distancia_min < 0.1:
    print("✅ Novo livro é similar a casos conhecidos — previsão confiável")
elif distancia_min < 0.3:
    print("⚠️ Novo livro é parcialmente similar — previsão moderadamente confiável")
else:
    print("❌ Novo livro muito diferente — previsão incerta")

In [ ]:
# c) Visualização: previsões vs. reais
# Um gráfico rápido para inspecionar se o modelo é consistente:
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--")
plt.xlabel("Vendas reais")
plt.ylabel("Vendas previstas")
plt.title("Acurácia das previsões por autor/livraria")
plt.show()

In [ ]:
# 🧩 4. Dica prática: criar uma “faixa de previsão”
# Você pode combinar a média e o desvio entre árvores para dar uma faixa de confiança, por exemplo:
ic_inferior = media - 1.96 * desvio
ic_superior = media + 1.96 * desvio

print(f"Intervalo de confiança 95%: {ic_inferior:.0f} a {ic_superior:.0f} cópias")